In [1]:
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from PIL import Image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

**Loading Dataset**
$\newline$
In my case there are (28,28) images of English letters, each column from 0 to 783 represents one pixel
$\newline$
'labels' columns represents class (English letter)

In [2]:
df=pd.read_csv('ALPHABET.csv',index_col=0)
df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,labels
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372445,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z
372446,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z
372447,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z
372448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z


**Each class has different number of lines so we have to cut it**

In [3]:
print(dict(df['labels'].value_counts().items()))

{'o': 57825, 's': 48419, 'u': 29008, 'c': 23409, 't': 22495, 'p': 19341, 'n': 19010, 'a': 13869, 'm': 12336, 'l': 11586, 'r': 11566, 'e': 11440, 'y': 10859, 'w': 10784, 'd': 10134, 'b': 8668, 'j': 8493, 'h': 7218, 'x': 6272, 'z': 6076, 'q': 5812, 'g': 5762, 'k': 5603, 'v': 4182, 'f': 1163, 'i': 1120}


In [4]:
grouped = df.groupby('labels')

trimmed_data = []

for name, group in grouped:
    group = group.sample(frac=1, random_state=42)
    trimmed_group = group.head(1000)
    trimmed_data.append(trimmed_group)
trimmed_df = pd.concat(trimmed_data)
trimmed_df

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,labels
2334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
1078,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
6647,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
6967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
5044,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368214,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z
371709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z
370196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z
367122,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,z


In [5]:
print(dict(trimmed_df['labels'].value_counts().items()))

{'a': 1000, 'b': 1000, 'y': 1000, 'x': 1000, 'w': 1000, 'v': 1000, 'u': 1000, 't': 1000, 's': 1000, 'r': 1000, 'q': 1000, 'p': 1000, 'o': 1000, 'n': 1000, 'm': 1000, 'l': 1000, 'k': 1000, 'j': 1000, 'i': 1000, 'h': 1000, 'g': 1000, 'f': 1000, 'e': 1000, 'd': 1000, 'c': 1000, 'z': 1000}


**Preprocessing data for learning**

In [6]:
X = trimmed_df.drop(columns=['labels'])
y = trimmed_df['labels']
X=X.to_numpy().astype('uint8')
y=y.to_numpy()
X=X.reshape(-1,28,28)

**Making mappings to conver labels to ids**

In [7]:
decode_map={}
last=0
for i1 in y:
    if i1 not in decode_map:
        decode_map[i1]=last
        last+=1
y = np.array([decode_map[i] for i in y])
y=to_categorical(y)

In [8]:
print(decode_map)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'q': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (20800, 28, 28)
X_test shape: (5200, 28, 28)
y_train shape: (20800, 26)
y_test shape: (5200, 26)


**Now we need to add one more dimension to X**

In [10]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (20800, 28, 28, 1)
X_test shape: (5200, 28, 28, 1)
y_train shape: (20800, 26)
y_test shape: (5200, 26)


**Creating model using keras**

In [11]:
def create_model():
    model = Sequential()
    model.add(Conv2D(64, (2, 2), activation='relu', input_shape=(28,28,1)))
    model.add(Dropout(0.1))
    model.add(Conv2D(128, (3, 3), activation='relu', ))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(26, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

**Creating datagen to use data augmentation**

In [12]:
datagen = ImageDataGenerator(rotation_range=15,
                             width_shift_range=0.15,
                             height_shift_range=0.15,
                             shear_range=0.15,
                             zoom_range=0.25,
                             fill_mode='nearest')

**Creating an early stopping callback, so model will continue to learn as long as loss decreases**

In [13]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

**Creating a model and train it**

In [14]:
model=create_model()
model.fit(datagen.flow(X_train, y_train, batch_size=16),
        validation_data=(X_test, y_test),
        epochs=100,
        callbacks=[early_stopping_callback])

test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)




Epoch 1/100


1300/1300 [==============================] - 63s 48ms/step - loss: 1.5556 - accuracy: 0.5217 - val_loss: 0.3415 - val_accuracy: 0.9046
Epoch 2/100
1300/1300 [==============================] - 64s 49ms/step - loss: 0.8001 - accuracy: 0.7492 - val_loss: 0.2216 - val_accuracy: 0.9398
Epoch 3/100
1300/1300 [==============================] - 72s 55ms/step - loss: 0.6104 - accuracy: 0.8125 - val_loss: 0.2275 - val_accuracy: 0.9321
Epoch 4/100
1300/1300 [==============================] - 69s 53ms/step - loss: 0.5313 - accuracy: 0.8360 - val_loss: 0.1900 - val_accuracy: 0.9442
Epoch 5/100
1300/1300 [==============================] - 72s 55ms/step - loss: 0.4842 - accuracy: 0.8530 - val_loss: 0.1626 - val_accuracy: 0.9550
Epoch 6/100
1300/1300 [==============================] - 73s 56ms/step - loss: 0.4367 - accuracy: 0.8655 - val_loss: 0.1438 - val_accuracy: 0.9598
Epoch 7/100
1300/1300 [==============================] - 75s 58ms/step - loss: 0.4107 - accuracy: 0.8760 - val_lo

**Scoring**

In [15]:
y_pred=model.predict(X_test)
r2=r2_score(y_test,y_pred)
print(f'r2_score: {r2}')

163/163 [==============================] - 5s 32ms/step
r2_score: 0.9397262188651697


**Saving the model**

In [16]:
model.save(f'ENG{round(test_acc,2)}.h5')

C:\Users\AI_DEV\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
print(y_train[0])
Image.fromarray((X_train[0].reshape(28,28)*255).astype('uint8'))

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
